In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import spacy
import json
import random
import re

In [3]:
#function to load the data

def load_data(file):
    with open(file, "r", encoding="utf-8") as f:
        data = json.load(f)
    return (data)

In [4]:
#function to save the data

def save_data(file, data):
    with open(file, "w", encoding="uft-8") as f:
        json.dump(data, f, indent=4)

In [5]:
#function to train the data

def train_spacy(data, iterations):
    TRAIN_DATA = data

    #loading the blank model to train
    nlp = spacy.blank("en")

    #adding ner pipe to loaded model
    if "ner" not in nlp.pipe_names:
        ner = nlp.create_pipe("ner")
        nlp.add_pipe(ner, last=True)

    #adding label to the model in pipe
    for _, annotations in TRAIN_DATA:
        for ent in annotations.get("entities"):
            ner.add_label(ent[2])
    
    #disabling other pipes to model
    other_pipes = [pipe for pipe in nlp.pipe_names if pipe != "ner"]
    with nlp.disable_pipes(*other_pipes):

      #training begin to the model
        optimizer = nlp.begin_training()
        for itn in range(iterations):
            print ("Starting iteration " + str(itn))
            random.shuffle(TRAIN_DATA)
            losses = {}
            for text, annotations in TRAIN_DATA:
                nlp.update(
                            [text],
                            [annotations],
                            drop=0.2,
                            sgd=optimizer,
                            losses=losses
                )
            print (losses)
    return (nlp)

In [6]:
#function to clean the data

def clean_text(text):
  cleaned = re.sub(r"[\(\[].*?[\)\]]", "", text)
  return (cleaned)

In [7]:
#read the data from json file

TRAIN_DATA = load_data("/content/drive/MyDrive/Phase1/noun_d1.json")

In [8]:
TRAIN_DATA

[['I want to lose weight, yet I eat chocolate daily.',
  {'entities': [[15, 21, 'NOUN'], [33, 42, 'NOUN']]}],
 ["He eats macaroni, but he won't eat cheese.",
  {'entities': [[7, 15, 'NOUN'], [33, 39, 'NOUN']]}],
 ['I hate spinach; however,I love broccoli.',
  {'entities': [[6, 13, 'NOUN'], [25, 29, 'NOUN'], [30, 38, 'NOUN']]}],
 ["I don't want to go outtonight; besides, I have homework to do.",
  {'entities': [[22, 29, 'NOUN'], [47, 55, 'NOUN']]}],
 ['I want to graduate withhonors; furthermore,I want to go to law school.',
  {'entities': [[10, 18, 'NOUN'], [23, 29, 'NOUN'], [59, 69, 'NOUN']]}],
 ['My brother just graduated from highschool,sohe will attend St. Petersburg College.',
  {'entities': [[3, 10, 'NOUN'], [35, 41, 'NOUN'], [59, 81, 'NOUN']]}],
 ['I was tired from workinglate,but Ihad to go to class anyway.',
  {'entities': [[11, 14, 'NOUN'], [42, 47, 'NOUN']]}],
 ['She had togo, soshe called a friend to drive her. ',
  {'entities': [[29, 35, 'NOUN'], [39, 44, 'NOUN']]}],
 ['Gil

In [9]:
#call the train function with iterations

nlp = train_spacy(TRAIN_DATA, 30)
nlp.to_disk("/content/drive/MyDrive/Phase1/noun_model")

Starting iteration 0
{'ner': 323.212132299019}
Starting iteration 1
{'ner': 80.06678543548736}
Starting iteration 2
{'ner': 61.49183880126971}
Starting iteration 3
{'ner': 48.90931739134372}
Starting iteration 4
{'ner': 17.86675898536386}
Starting iteration 5
{'ner': 23.195030274028294}
Starting iteration 6
{'ner': 18.65287612557174}
Starting iteration 7
{'ner': 12.3951591980296}
Starting iteration 8
{'ner': 16.885631046922075}
Starting iteration 9
{'ner': 12.383151814174173}
Starting iteration 10
{'ner': 15.55000599497789}
Starting iteration 11
{'ner': 17.788543145128173}
Starting iteration 12
{'ner': 18.769320327709924}
Starting iteration 13
{'ner': 19.79629527823036}
Starting iteration 14
{'ner': 38.19587045545842}
Starting iteration 15
{'ner': 29.433649696227008}
Starting iteration 16
{'ner': 22.53409374671416}
Starting iteration 17
{'ner': 22.54240465226587}
Starting iteration 18
{'ner': 19.38052212517648}
Starting iteration 19
{'ner': 13.391263601939864}
Starting iteration 20
{'n

In [48]:
#reading the fresh data

test = "My name is tiger"

In [49]:
#clean_text calls the clean function and removes all unwanted symbols

test = clean_text(test)

In [50]:
#loading the created model

nlp = spacy.load("/content/drive/MyDrive/Phase1/noun_model")

In [51]:
doc = nlp(test)

In [52]:
#printing the entities

for ent in doc.ents:
  print (ent.text, ent.label_)

In [15]:
from spacy.gold import GoldParse
from spacy.scorer import Scorer

In [16]:
#
#def evaluate(ner_model, TRAIN_DATA):
#    scorer = Scorer()
#    for input_, annot in TRAIN_DATA:
#        doc_gold_text = ner_model.make_doc(input_)
#        gold = GoldParse(doc_gold_text, entities=annot)
#        pred_value = ner_model(input_)
#        scorer.score(pred_value, gold)
#    return scorer.scores

In [17]:
#ner_model = spacy.load("/content/drive/MyDrive/Noun-json/nlp_model") 
#results = evaluate(ner_model, TRAIN_DATA)

In [18]:
#results

In [19]:
import random
import logging
from sklearn.metrics import classification_report
from sklearn.metrics import precision_recall_fscore_support
from spacy.gold import GoldParse
from spacy.scorer import Scorer
from sklearn.metrics import accuracy_score
import re
import spacy
from sklearn import metrics

In [20]:

def trim_entity_spans(data: list) -> list:
    invalid_span_tokens = re.compile(r'\s')
    cleaned_data = []
    for text, annotations in TRAIN_DATA:
        entities = annotations['entities']
        valid_entities = []
        for start, end, label in entities:
            valid_start = start
            valid_end = end
            while valid_start < len(text) and invalid_span_tokens.match(
                    text[valid_start]):
                valid_start += 1
            while valid_end > 1 and invalid_span_tokens.match(
                    text[valid_end - 1]):
                valid_end -= 1
            valid_entities.append([valid_start, valid_end, label])
        cleaned_data.append([text, {'entities': valid_entities}])
    return cleaned_data


In [21]:

def convert_dataturks_to_spacy(dataturks_JSON_FilePath):
    try:
        training_data = []
        lines=[]
        with open(dataturks_JSON_FilePath, encoding="utf-8") as f:
            lines = f.readlines()            
        for line in lines:
            data = json.loads(line)
            text = data['content']
            entities = []
            for annotation in data['annotation']:
                point = annotation['points'][0]
                labels = annotation['label']
                if not isinstance(labels, list):
                    labels = [labels]

                for label in labels:
                    entities.append((point['start'], point['end'] + 1 ,label))
            training_data.append((text, {"entities" : entities}))
        return training_data
    except Exception as e:
        #logging.exception("Unable to process " + dataturks_JSON_FilePath + "\n" + "error = " + str(e))
        return None



In [22]:
def train_spacy():
    TRAIN_DATA = convert_dataturks_to_spacy("/content/drive/MyDrive/Phase1/noun_d1.json")
    TRAIN_DATA = trim_entity_spans(TRAIN_DATA)
    examples = convert_dataturks_to_spacy("/content/drive/MyDrive/Phase1/noun_d1.json")
    examples = trim_entity_spans(examples)
    tp=0
    tr=0
    tf=0
    ta=0
    c=0        
    for text,annot in examples:
        f=open("resume"+str(c)+".txt","w", encoding ='utf-8')
        nlp_trained = spacy.load("/content/drive/MyDrive/Phase1/noun_model")
        doc_to_test=nlp_trained(text)
        d={}
        for ent in doc_to_test.ents:
            d[ent.label_]=[]
        for ent in doc_to_test.ents:
            d[ent.label_].append(ent.text)
        for i in set(d.keys()):
            f.write("\n\n")
            f.write(i +":"+"\n")
            for j in set(d[i]):
                f.write(j.replace('\n','')+"\n")
        d={}
        for ent in doc_to_test.ents:
            d[ent.label_]=[0,0,0,0,0,0]
        for ent in doc_to_test.ents:
            doc_gold_text= nlp_trained.make_doc(text)
            gold = GoldParse(doc_gold_text, entities=annot.get("entities"))
            y_true = [ent.label_ if ent.label_ in x else 'Not '+ent.label_ for x in gold.ner]
            y_pred = [x.ent_type_ if x.ent_type_ ==ent.label_ else 'Not '+ent.label_ for x in doc_to_test]
            #print(metrics.confusion_matrix(y_true, y_pred))
            #print(metrics.classification_report(y_true, y_pred))  
            if(d[ent.label_][0]==0):
                (p,r,f,s)= precision_recall_fscore_support(y_true,y_pred,average='weighted')
                a=accuracy_score(y_true,y_pred)
                d[ent.label_][0]=1
                d[ent.label_][1]+=p
                d[ent.label_][2]+=r
                d[ent.label_][3]+=f
                d[ent.label_][4]+=a
                d[ent.label_][5]+=1
           
        c+=1
        
    for i in d:
        print("\n For Entity "+i+"\n")
        print("Accuracy : "+str((d[i][4]/d[i][5])*100)+"%")
        print("Precision : "+str(d[i][1]/d[i][5]))
        print("Recall : "+str(d[i][2]/d[i][5]))
        print("F-score : "+str(d[i][3]/d[i][5]))


In [24]:
train_spacy()

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))



 For Entity NOUN

Accuracy : 90.0%
Precision : 1.0
Recall : 0.9
F-score : 0.9473684210526316


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
